In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [13]:
factor_stack = pd.read_pickle("./data/processed_data/factor_stack_2018_2019_test.pkl")
factor_stack.index = factor_stack.index.swaplevel(0,1)
factor_stack.sort_index(level=0, inplace=True)
factor_stack.head()

alphaP0001  alphaP0002  alphaP0003  alphaP0004  \
code      date                                                         
000001.SZ 2018-01-02    2.092775    2.308255    0.329311    0.619233   
          2018-01-03    2.657483    2.323697    0.308489    0.878148   
          2018-01-04   -0.029540    2.287080    0.343293    0.122582   
          2018-01-05    0.316773    2.250931    0.264644   -0.121824   
          2018-01-08    1.658981    2.225216    0.153681    0.517513   

                      alphaP0005  alphaP0006  alphaP0007  alphaP0008  \
code      date                                                         
000001.SZ 2018-01-02    0.230993    0.927089   -0.303216    1.029343   
          2018-01-03    0.278084   -1.966891   -0.329762   -2.176700   
          2018-01-04    0.095521    0.247447   -0.333903   -1.703234   
          2018-01-05    0.294053   -0.693630   -0.329043    0.594335   
          2018-01-08    0.398156   -1.735062   -0.342388   -2.043304   

                      alphaP0009  alphaP0010  ...  alphaP2019_120_60  \
code      date                                ...                      
000001.SZ 2018-01-02    1.218699    0.000000  ...           1.020801   
          2018-01-03    0.000000    0.000000  ...           1.122253   
          2018-01-04    0.000000    0.651004  ...           1.168328   
          2018-01-05    0.000000   -0.880260  ...           1.153758   
          2018-01-08    0.000000   -0.231624  ...           1.209609   

                      alphaP2019_200_10  alphaP2019_200_120  \
code      date                                                
000001.SZ 2018-01-02          -1.275379            0.415361   
          2018-01-03          -1.277108            0.500440   
          2018-01-04          -1.322103            0.523787   
          2018-01-05          -1.422960            0.507992   
          2018-01-08          -1.120461            0.569762   

                      alphaP2019_200_20  alphaP2019_200_200  alphaP2019_200_5  \
code      date                                                                  
000001.SZ 2018-01-02          -0.677882            0.485807         -1.548870   
          2018-01-03          -0.528526            0.579212         -1.548925   
          2018-01-04          -0.520922            0.615010         -1.499024   
          2018-01-05          -0.623765            0.609276         -1.540481   
          2018-01-08          -0.489789            0.665397         -1.062404   

                      alphaP2019_200_60  alphaP2019_20_10  alphaP2019_20_120  \
code      date                                                                 
000001.SZ 2018-01-02           0.972719          0.002871           0.460742   
          2018-01-03           1.069962          0.244758           0.551550   
          2018-01-04           1.112668          0.137152           0.578172   
          2018-01-05           1.099127         -0.043438           0.561162   
          2018-01-08           1.151547          0.517523           0.625696   

                      alphaP2019_20_20  
code      date                          
000001.SZ 2018-01-02          0.086009  
          2018-01-03          0.285841  
          2018-01-04          0.293727  
          2018-01-05          0.184161  
          2018-01-08          0.349869  

[5 rows x 975 columns]

In [35]:
valid_date = factor_stack.notna().all(axis=1)
valid_date = valid_date.groupby('code', group_keys=False).rolling(5).sum().droplevel(0)
valid_date = np.array(valid_date[valid_date==5].index)
valid_date[:5]

array([('000001.SZ', Timestamp('2018-01-08 00:00:00')),
       ('000001.SZ', Timestamp('2018-01-09 00:00:00')),
       ('000001.SZ', Timestamp('2018-01-10 00:00:00')),
       ('000001.SZ', Timestamp('2018-01-11 00:00:00')),
       ('000001.SZ', Timestamp('2018-01-12 00:00:00'))], dtype=object)